In [90]:
import pandas as pd

In [91]:
df = pd.read_csv('s3://fintech540-ml/external/layoffs_data.csv')

In [92]:
df.head()

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off
0,IRL,SF Bay Area,Consumer,1.00,2023-06-23,https://www.theinformation.com/articles/social...,197.0,Series C,2023-06-25 23:04:16,United States,NaN,Unknown
1,Retool,SF Bay Area,Other,0.09,2023-06-22,Internal memo,141.0,Series C,2023-06-22 14:23:40,United States,NaN,Unknown
2,Anaplan,SF Bay Area,Other,NaN,2023-06-21,https://nypost.com/2023/06/23/software-giant-a...,300.0,Acquired,2023-06-24 03:48:00,United States,300.0,Unknown
3,Uber,SF Bay Area,Transportation,NaN,2023-06-21,https://www.nasdaq.com/articles/uber-to-lay-of...,25200.0,Post-IPO,2023-06-22 00:19:03,United States,200.0,Unknown
4,Tackle.io,Boise,Infrastructure,NaN,2023-06-21,https://tackle.io/blog/tackle-company-update/,148.0,Series C,2023-06-23 15:38:59,United States,75.0,Unknown


In [93]:
df['Date'] = pd.to_datetime(df['Date'])

df = (df.groupby(['Industry', 'Date'])
        .agg({'Funds_Raised': ['mean', 'sum'], 
              'Laid_Off_Count': ['mean', 'sum']})
        .reset_index())

df.columns = ['industry', 'date', 'funds_raised_mean', 'funds_raised_sum', 
              'laid_off_count_mean', 'laid_off_count_sum']

df = df.fillna(0)

In [94]:
df['industry'] = df['industry'].str.lower()

In [95]:
df_pivoted = df.pivot(index='date', columns='industry')

# Flatten the MultiIndex columns and join with the industry names
df_pivoted.columns = ['{}_{}'.format(industry, metric) for metric, industry in df_pivoted.columns]

# Reset index to make 'date' a column again
df_pivoted.reset_index(inplace=True)

# Display the transformed DataFrame
df_pivoted.columns

Index(['date', 'aerospace_funds_raised_mean', 'construction_funds_raised_mean',
       'consumer_funds_raised_mean', 'crypto_funds_raised_mean',
       'data_funds_raised_mean', 'education_funds_raised_mean',
       'energy_funds_raised_mean', 'finance_funds_raised_mean',
       'fitness_funds_raised_mean',
       ...
       'other_laid_off_count_sum', 'product_laid_off_count_sum',
       'real estate_laid_off_count_sum', 'recruiting_laid_off_count_sum',
       'retail_laid_off_count_sum', 'sales_laid_off_count_sum',
       'security_laid_off_count_sum', 'support_laid_off_count_sum',
       'transportation_laid_off_count_sum', 'travel_laid_off_count_sum'],
      dtype='object', length=117)

In [96]:
df_pivoted

,date,aerospace_funds_raised_mean,construction_funds_raised_mean,consumer_funds_raised_mean,crypto_funds_raised_mean,data_funds_raised_mean,education_funds_raised_mean,energy_funds_raised_mean,finance_funds_raised_mean,fitness_funds_raised_mean,...,other_laid_off_count_sum,product_laid_off_count_sum,real estate_laid_off_count_sum,recruiting_laid_off_count_sum,retail_laid_off_count_sum,sales_laid_off_count_sum,security_laid_off_count_sum,support_laid_off_count_sum,transportation_laid_off_count_sum,travel_laid_off_count_sum
0,2020-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN
2,2020-03-13,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN
3,2020-03-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,130.0
4,2020-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2023-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,2023-06-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,NaN
571,2023-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,NaN
572,2023-06-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
df_filled = df_pivoted.fillna(0)

# Step 2: Ensure all dates are represented
# Convert the 'date' column to datetime if it's not already
df_filled['date'] = pd.to_datetime(df_filled['date'])

# Create a date range that covers your entire period
date_range = pd.date_range(start=df_filled['date'].min(), end=df_filled['date'].max())

# Reindex the DataFrame using this date range
df_filled.set_index('date', inplace=True)
df_filled = df_filled.reindex(date_range, fill_value=0).reset_index()

# Rename the 'index' column back to 'date'
df_filled.rename(columns={'index': 'date'}, inplace=True)

# Display the transformed DataFrame
print(df_filled)

           date  aerospace_funds_raised_mean  construction_funds_raised_mean  \
0    2020-03-11                          0.0                             0.0   
1    2020-03-12                          0.0                             0.0   
2    2020-03-13                          0.0                             0.0   
3    2020-03-14                          0.0                             0.0   
4    2020-03-15                          0.0                             0.0   
...         ...                          ...                             ...   
1195 2023-06-19                          0.0                             0.0   
1196 2023-06-20                          0.0                             0.0   
1197 2023-06-21                          0.0                             0.0   
1198 2023-06-22                          0.0                             0.0   
1199 2023-06-23                          0.0                             0.0   

      consumer_funds_raised_mean  crypt

In [98]:
df_filled.to_csv('INDUSTRY_LAYOFFS_DAILY.csv', index=False)